![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

# Building on Your RAG Agent: Adding Memory for Context Engineering

## From Grounding Problem to Memory Solution

In the previous notebook, you experienced the **grounding problem** - how references break without memory. Now you'll enhance your existing RAG agent from Section 2 with memory capabilities.

### What You'll Build

**Enhance your existing `SimpleRAGAgent`** with memory:

- **🧠 Working Memory** - Session-scoped conversation context
- **📚 Long-term Memory** - Cross-session knowledge and preferences  
- **🔄 Memory Integration** - Seamless working + long-term memory
- **⚡ Agent Memory Server** - Production-ready memory architecture

### Context Engineering Focus

This notebook teaches **memory-enhanced context engineering** by building on your existing agent:

1. **Reference Resolution** - Using memory to resolve pronouns and references
2. **Memory-Aware Context Assembly** - How memory improves context quality
3. **Personalized Context** - Leveraging long-term memory for personalization
4. **Cross-Session Continuity** - Context that survives across conversations

### Learning Objectives

By the end of this notebook, you will:
1. **Enhance** your existing RAG agent with memory capabilities
2. **Implement** working memory for conversation context
3. **Use** long-term memory for persistent knowledge
4. **Build** memory-enhanced context engineering patterns
5. **Create** a final production-ready memory-enhanced agent class

## Setup: Import Components and Initialize Environment

Let's start by importing your RAG agent from Section 2 and the memory components we'll use to enhance it.

### 🎯 **What We're Importing**
- **Your RAG agent models** from Section 2 (`StudentProfile`, `Course`, etc.)
- **Course manager** for searching Redis University courses
- **LangChain components** for LLM interaction
- **Agent Memory Server client** for production-ready memory

In [1]:
# Setup: Import your RAG agent and memory components
import os
import sys
import asyncio
from typing import List, Dict, Any, Optional
from datetime import datetime
from dotenv import load_dotenv

# Load environment
load_dotenv()
sys.path.append('../../reference-agent')

# Import your RAG agent components from Section 2
from redis_context_course.models import (
    Course, StudentProfile, DifficultyLevel, 
    CourseFormat, Semester
)
from redis_context_course.course_manager import CourseManager
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

# Import Agent Memory Server client
try:
    from agent_memory_client import MemoryAPIClient, MemoryClientConfig
    from agent_memory_client.models import WorkingMemory, MemoryMessage
    MEMORY_SERVER_AVAILABLE = True
    print("✅ Agent Memory Server client available")
except ImportError:
    MEMORY_SERVER_AVAILABLE = False
    print("⚠️  Agent Memory Server not available")
    print("📝 Install with: pip install agent-memory-client")
    print("🚀 Start server with: docker-compose up")

# Verify environment
if not os.getenv("OPENAI_API_KEY"):
    print("❌ OPENAI_API_KEY not found. Please set in .env file.")
else:
    print("✅ OPENAI_API_KEY found")

print(f"\n🔧 Environment Setup:")
print(f"   OPENAI_API_KEY: {'✓ Set' if os.getenv('OPENAI_API_KEY') else '✗ Not set'}")
print(f"   AGENT_MEMORY_URL: {os.getenv('AGENT_MEMORY_URL', 'http://localhost:8088')}")
print(f"   Memory Server: {'✓ Available' if MEMORY_SERVER_AVAILABLE else '✗ Not available'}")

✅ Agent Memory Server client available
✅ OPENAI_API_KEY found

🔧 Environment Setup:
   OPENAI_API_KEY: ✓ Set
   AGENT_MEMORY_URL: http://localhost:8088
   Memory Server: ✓ Available


### 🎯 **What We Just Did**

**Successfully Imported:**
- ✅ **Your RAG agent models** from Section 2
- ✅ **Agent Memory Server client** for production-ready memory
- ✅ **Environment verified** - OpenAI API key and memory server ready

**Why This Matters:**
- We're building **on top of your existing Section 2 foundation**
- **Agent Memory Server** provides scalable, persistent memory (vs simple in-memory storage)
- **Production-ready architecture** that can handle real applications

**Next:** We'll recreate your `SimpleRAGAgent` from Section 2 as our starting point.

## Step 1: Your RAG Agent from Section 2

Let's start with your `SimpleRAGAgent` from Section 2. This is the foundation we'll enhance with memory.

### 🔍 **Current Limitations (What We'll Fix)**
- **Session-bound memory** - Forgets everything when restarted
- **No reference resolution** - Can't understand "it", "that", "you mentioned"
- **Limited conversation history** - Only keeps last 2 messages
- **No personalization** - Doesn't learn student preferences

### 🚀 **What We'll Add**
- **Working memory** - Persistent conversation context for reference resolution
- **Long-term memory** - Cross-session knowledge and preferences
- **Memory-enhanced context** - Smarter context assembly using memory

In [2]:
# Your SimpleRAGAgent from Section 2 - the foundation we'll enhance
class SimpleRAGAgent:
    """Your RAG agent from Section 2 - foundation for memory enhancement"""
    
    def __init__(self, course_manager: CourseManager):
        self.course_manager = course_manager
        self.llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0.7)
        self.conversation_history = {}  # In-memory only - lost when restarted!
    
    async def search_courses(self, query: str, limit: int = 3) -> List[Course]:
        """Search for relevant courses using the course manager"""
        results = await self.course_manager.search_courses(query, limit=limit)
        return results
    
    def create_context(self, student: StudentProfile, query: str, courses: List[Course]) -> str:
        """Create context for the LLM - your excellent context engineering from Section 2"""
        
        # Student context
        student_context = f"""STUDENT PROFILE:
Name: {student.name}
Academic Status: {student.major}, Year {student.year}
Completed Courses: {', '.join(student.completed_courses) if student.completed_courses else 'None'}
Learning Interests: {', '.join(student.interests)}
Preferred Format: {student.preferred_format.value if student.preferred_format else 'Any'}"""
        
        # Courses context
        courses_context = "RELEVANT COURSES:\n"
        for i, course in enumerate(courses, 1):
            courses_context += f"{i}. {course.course_code}: {course.title}\n"
        
        # Basic conversation history (limited and session-bound)
        history_context = ""
        if student.email in self.conversation_history:
            history = self.conversation_history[student.email]
            if history:
                history_context = "\nRECENT CONVERSATION:\n"
                for msg in history[-2:]:  # Only last 2 messages
                    history_context += f"User: {msg['user']}\nAssistant: {msg['assistant']}\n"
        
        return student_context + "\n\n" + courses_context + history_context
    
    async def chat(self, student: StudentProfile, query: str) -> str:
        """Chat with the student using RAG"""
        relevant_courses = await self.search_courses(query, limit=3)
        context = self.create_context(student, query, relevant_courses)
        
        system_message = SystemMessage(content="""You are a helpful academic advisor for Redis University. 
Use the provided context to give personalized course recommendations.
Be specific and explain why courses are suitable for the student.""")
        
        human_message = HumanMessage(content=f"Context: {context}\n\nStudent Question: {query}")
        response = self.llm.invoke([system_message, human_message])
        
        # Store in basic memory (session-bound)
        if student.email not in self.conversation_history:
            self.conversation_history[student.email] = []
        
        self.conversation_history[student.email].append({
            "user": query,
            "assistant": response.content
        })
        
        return response.content

print("📝 SimpleRAGAgent defined (your Section 2 foundation)")
print("❌ Limitations: Session-bound memory, no reference resolution, limited context")

📝 SimpleRAGAgent defined (your Section 2 foundation)
❌ Limitations: Session-bound memory, no reference resolution, limited context


### 🎯 **What We Just Built**

**Your `SimpleRAGAgent` from Section 2:**
- ✅ **Course search** - Finds relevant courses using vector search
- ✅ **Context engineering** - Assembles student profile + courses + basic history
- ✅ **LLM interaction** - Gets personalized responses from GPT
- ✅ **Basic memory** - Stores conversation in Python dictionary

**Current Problems (The Grounding Problem):**
- ❌ **"What are its prerequisites?"** → Agent doesn't know what "its" refers to
- ❌ **"Can I take it?"** → Agent doesn't know what "it" refers to
- ❌ **Session-bound** - Memory lost when restarted
- ❌ **Limited history** - Only last 2 messages

**Next:** We'll add persistent memory to solve these problems.

## Step 2: Initialize Memory Client

Now let's set up the Agent Memory Server client that will provide persistent memory capabilities.

### 🧠 **What Agent Memory Server Provides**
- **Working Memory** - Session-scoped conversation context (solves grounding problem)
- **Long-term Memory** - Cross-session knowledge and preferences
- **Semantic Search** - Vector-based memory retrieval
- **Automatic Extraction** - AI extracts important facts from conversations
- **Production Scale** - Redis-backed, handles thousands of users

In [3]:
# Initialize Memory Client for persistent memory
if MEMORY_SERVER_AVAILABLE:
    # Configure memory client
    config = MemoryClientConfig(
        base_url=os.getenv("AGENT_MEMORY_URL", "http://localhost:8088"),
        default_namespace="redis_university"
    )
    memory_client = MemoryAPIClient(config=config)
    
    print("🧠 Memory Client Initialized")
    print(f"   Base URL: {config.base_url}")
    print(f"   Namespace: {config.default_namespace}")
    print("   Ready for memory operations")
else:
    print("⚠️  Simulating memory operations (Memory Server not available)")
    memory_client = None

🧠 Memory Client Initialized
   Base URL: http://localhost:8088
   Namespace: redis_university
   Ready for memory operations
